In [4]:
import sys
print(sys.executable)

c:\Users\misba\OneDrive\Desktop\LM_Project\.venv\Scripts\python.exe


In [9]:
# LM Project — GPT-2 Fine-tuning and Analysis
Author: Misbah
Date: 18/10/2025
Objective: Fine-tune GPT-2 on domain text, analyze generation quality, attention, and decoding behaviour. Run experiments, create visualizations, and save results.


SyntaxError: invalid syntax (3720596467.py, line 4)

In [10]:
import os, random, math, time
import numpy as np, pandas as pd
import matplotlib.pyplot as plt, seaborn as sns
import torch
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM

# Versions & device
print("Python:", os.sys.version.split()[0])
print("torch:", torch.__version__)
print("transformers:", AutoModelForCausalLM.__module__)

device = "cuda" if torch.cuda.is_available() else "cpu"
print("DEVICE:", device)

# Reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if device == "cuda":
    torch.cuda.manual_seed_all(seed)


c:\Users\misba\OneDrive\Desktop\LM_Project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Python: 3.12.10
torch: 2.9.0+cpu
transformers: transformers.models.auto.modeling_auto
DEVICE: cpu


In [11]:
# Load a small dataset (wikitext-2) for quick testing
ds = load_dataset("wikitext", "wikitext-2-raw-v1")
train_texts = [t for t in ds['train']['text'] if t.strip()]

# Quick subset for first experiments
train_texts = train_texts[:10000]
print("Number of training lines:", len(train_texts))


c:\Users\misba\OneDrive\Desktop\LM_Project\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\misba\.cache\huggingface\hub\datasets--wikitext. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP 

Number of training lines: 10000


In [12]:
model_name = "gpt2"  # small GPT-2 model
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # necessary for padding

# Load pretrained GPT-2
baseline_model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

# Test generation
prompt = "The future of AI in education is"
inputs = tokenizer(prompt, return_tensors="pt").to(device)
baseline_output = baseline_model.generate(**inputs, max_new_tokens=60)
print("Baseline generation:\n", tokenizer.decode(baseline_output[0], skip_special_tokens=True))


c:\Users\misba\OneDrive\Desktop\LM_Project\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\misba\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP downlo

Baseline generation:
 The future of AI in education is uncertain. The future of AI in education is uncertain.

The future of AI in education is uncertain.

The future of AI in education is uncertain.

The future of AI in education is uncertain.

The future of AI in education is uncertain.

The future of


In [13]:
from datasets import Dataset

# Convert to HuggingFace Dataset
hf_ds = Dataset.from_dict({"text": train_texts})

# Tokenization
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=256)

tokenized_ds = hf_ds.map(tokenize_function, batched=True, remove_columns=["text"])
tokenized_ds = tokenized_ds.train_test_split(test_size=0.05, seed=seed)
print(tokenized_ds)


Map: 100%|██████████| 10000/10000 [00:00<00:00, 16990.31 examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 9500
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 500
    })
})


In [16]:
# ------------------------------
# Fine-tune GPT-2 on 10k samples
# ------------------------------

from transformers import AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
import torch

# Use device (GPU if available)
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Training on device:", device)

# Load GPT-2 model fresh
model_name = "gpt2"  # small model for local experiments
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

# Data collator for causal language modeling (no MLM)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Training arguments
training_args = TrainingArguments(
    output_dir="./models/gpt2-finetuned",
    overwrite_output_dir=True,
    num_train_epochs=1,                  # quick run for testing
    per_device_train_batch_size=2,       # reduce if out-of-memory
    per_device_eval_batch_size=2,
    evaluation_strategy="steps",
    eval_steps=200,
    logging_steps=50,
    save_steps=500,
    save_total_limit=2,
    learning_rate=5e-5,
    weight_decay=0.01,
    fp16=torch.cuda.is_available(),      # use mixed precision if GPU supports
    report_to=[],                        # disables W&B reporting
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    data_collator=data_collator,
)

# Start training
train_result = trainer.train()

# Save model and tokenizer
trainer.save_model("./models/gpt2-finetuned")
tokenizer.save_pretrained("./models/gpt2-finetuned")

# Print final results
train_result


Training on device: cpu


TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

In [17]:
import transformers
print(transformers.__version__)

4.57.1


In [ ]:
from transformers import AutoModelForCausalLM, DataCollatorForLanguageModeling, Trainer, TrainingArguments

# Load model fresh (GPT-2 small)
model_name = "gpt2"
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

# Data collator for causal LM (no MLM)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Training arguments (works with transformers 4.57.1)
training_args = TrainingArguments(
    output_dir="./models/gpt2-finetuned",
    overwrite_output_dir=True,
    num_train_epochs=1,                  # quick run for testing
    per_device_train_batch_size=2,       # reduce if OOM
    per_device_eval_batch_size=2,
    evaluation_strategy="steps",
    eval_steps=200,
    logging_steps=50,
    save_steps=500,
    save_total_limit=2,
    learning_rate=5e-5,
    weight_decay=0.01,
    fp16=torch.cuda.is_available(),      # mixed precision if GPU
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    data_collator=data_collator,
)

# Start training
train_result = trainer.train()

# Save model + tokenizer
trainer.save_model("./models/gpt2-finetuned")
tokenizer.save_pretrained("./models/gpt2-finetuned")

train_result
